In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
import gresearch_crypto
from pathlib import Path
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/g-research-crypto-forecasting/example_sample_submission.csv
/kaggle/input/g-research-crypto-forecasting/asset_details.csv
/kaggle/input/g-research-crypto-forecasting/example_test.csv
/kaggle/input/g-research-crypto-forecasting/train.csv
/kaggle/input/g-research-crypto-forecasting/supplemental_train.csv
/kaggle/input/g-research-crypto-forecasting/gresearch_crypto/competition.cpython-37m-x86_64-linux-gnu.so
/kaggle/input/g-research-crypto-forecasting/gresearch_crypto/__init__.py


In [2]:
path = "/kaggle/input/g-research-crypto-forecasting/"
df_train = pd.read_csv(path + "train.csv")
df_test = pd.read_csv(path + "example_test.csv")
df_asset_details = pd.read_csv(path + "asset_details.csv")
df_supp_train = pd.read_csv(path + "supplemental_train.csv")

In [3]:
df_train.head()

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
0,1514764860,2,40.0,2376.5800,2399.5000,2357.1400,2374.5900,19.233005,2373.116392,-0.004218
1,1514764860,0,5.0,8.5300,8.5300,8.5300,8.5300,78.380000,8.530000,-0.014399
2,1514764860,1,229.0,13835.1940,14013.8000,13666.1100,13850.1760,31.550062,13827.062093,-0.014643
3,1514764860,5,32.0,7.6596,7.6596,7.6567,7.6576,6626.713370,7.657713,-0.013922
4,1514764860,7,5.0,25.9200,25.9200,25.8740,25.8770,121.087310,25.891363,-0.008264


In [4]:
df_train.isnull().sum()

timestamp         0
Asset_ID          0
Count             0
Open              0
High              0
Low               0
Close             0
Volume            0
VWAP              9
Target       750338
dtype: int64

In [5]:
df_train.columns

Index(['timestamp', 'Asset_ID', 'Count', 'Open', 'High', 'Low', 'Close',
       'Volume', 'VWAP', 'Target'],
      dtype='object')

In [6]:
df_train['datetime']=df_train.timestamp.astype('datetime64[s]')
df_test['datetime']=df_test.timestamp.astype('datetime64[s]')
df_train['time'] = df_train['datetime'].dt.time
df_train['date'] = df_train['datetime'].dt.date
df_test['time'] = df_test['datetime'].dt.time
df_test['date'] = df_test['datetime'].dt.date

In [7]:
cols=['Count', 'Open', 'High', 'Low', 'Close',
       'Volume']
def addAnalytics(dtFrame,funType,cols):
  if funType=='roll5':
    for i in cols:
      dtFrame[i.replace(' ','')+'_roll5']=dtFrame.sort_values(by=['date'])\
                       .groupby(['time'])[i]\
                       .rolling(5, min_periods = 1).mean()\
                       .reset_index()[i].round(3)
  if funType=='roll30':
    for i in cols:
      dtFrame[i.replace(' ','')+'_roll30']=dtFrame.sort_values(by=['date'])\
                       .groupby(['time'])[i]\
                       .rolling(10, min_periods = 1).mean()\
                       .reset_index()[i].round(3)

In [8]:
df_train.dropna(inplace=True)

In [9]:
# df_test.drop(['timestamp','datetime','time','date'],axis=1)
df_test.head()
# df_test.columns

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,group_num,row_id,datetime,time,date
0,1623542400,3,1201.0,1.478556,1.48603,1.47800,1.483681,654799.561103,1.481439,0,0,2021-06-13,00:00:00,2021-06-13
1,1623542400,2,1020.0,580.306667,583.89000,579.91000,582.276667,1227.988328,581.697038,0,1,2021-06-13,00:00:00,2021-06-13
2,1623542400,0,626.0,343.789500,345.10800,343.64000,344.598000,1718.832569,344.441729,0,2,2021-06-13,00:00:00,2021-06-13
3,1623542400,1,2888.0,35554.289632,35652.46465,35502.67000,35602.004286,163.811537,35583.469303,0,3,2021-06-13,00:00:00,2021-06-13
4,1623542400,4,433.0,0.312167,0.31260,0.31192,0.312208,585577.410442,0.312154,0,4,2021-06-13,00:00:00,2021-06-13


Reducing the data frame by choosing appropraite data types depending on the type of data not only decreases the modelling time also makes the use of computational resources appropriately. https://towardsdatascience.com/make-working-with-large-dataframes-easier-at-least-for-your-memory-6f52b5f4b5c4

In [10]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int8','int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [11]:
def getDataByAsset(dataFrameTrain,dataFrameTest,assetId):
    dataFrameTrain=dataFrameTrain[dataFrameTrain["Asset_ID"]==assetId]
    X=reduce_mem_usage(dataFrameTrain.drop(['Target','VWAP'],axis=1))
    y=dataFrameTrain['Target']
    addAnalytics(X,'roll5',cols)
    addAnalytics(X,'roll30',cols)
    X.index=X.datetime
    X.drop(['timestamp','datetime','time','date'],axis=1,inplace=True)
    X_test=reduce_mem_usage(dataFrameTest)
    addAnalytics(X_test,'roll5',cols)
    addAnalytics(X_test,'roll30',cols)
    X_test.index=X_test.datetime
    X_test=reduce_mem_usage(X_test[X.columns])
    min_max_scaler = MinMaxScaler()
    X = min_max_scaler.fit_transform(X)
    X_test = min_max_scaler.transform(X_test)
    return X,y,X_test

In [12]:
from sklearn.multioutput import MultiOutputRegressor
import lightgbm as lgb
!pip install optuna
import optuna

from sklearn.metrics import make_scorer, mean_absolute_error
from sklearn.metrics import mean_squared_error as MSE
class Objective():

    def __init__(self):
        self.best_booster = None
        self._booster = None

    def __call__(self, trial):
        param = {
            "objective": "regression",
            "metric": "mse",
            "verbosity": -1,
            "boosting_type": "gbdt",
            "feature_fraction": trial.suggest_loguniform("feature_fraction", 0.5, 1.0),
            "min_gain_to_split": trial.suggest_loguniform("min_gain_to_split", 0.3, 1.0),
            "max_depth": trial.suggest_int("max_depth", 20, 100),
            "lambda_l1": trial.suggest_loguniform("lambda_l1", 1e-8, 10.0),
            "lambda_l2": trial.suggest_loguniform("lambda_l2", 1e-8, 10.0),
            "num_leaves": trial.suggest_int("num_leaves", 40, 220),
            "num_iterations": trial.suggest_int("num_iterations", 20, 400),
            "learning_rate":trial.suggest_loguniform("learning_rate", 0.001, 0.1)
        }
        # Add a callback for pruning.
        pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "mse")
        model= lgb.LGBMRegressor()
        gbm = model.fit(X,y)

        self._booster = gbm

        preds = gbm.predict(X)
        pred_labels = np.rint(preds)
        accuracy = mean_absolute_error(y, pred_labels)
        return accuracy


In [13]:

from warnings import simplefilter
simplefilter("ignore", category=RuntimeWarning)
def getBooster():
    objective = Objective()
    optuna.logging.set_verbosity(optuna.logging.ERROR)
    study = optuna.create_study(
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=10), direction="minimize"
    )
    study.optimize(objective, n_trials=50)

    print("Best trial:")
    trial_co = study.best_trial

    print("  Params: ")
    for key, value in trial_co.params.items():
        print("    {}: {}".format(key, value))

    return objective.best_booster

In [14]:
from sklearn.metrics import make_scorer, mean_absolute_error
from sklearn.metrics import mean_squared_error as MSE
from sklearn.preprocessing import MinMaxScaler

In [15]:
X_dict = {}
y_dict = {}
model_dict = {}
preds_dict={}
for i in df_train.Asset_ID.unique():
    X,y,X_test=getDataByAsset(df_train,df_test,i)
    X_dict[i]=X
    y_dict[i]=y
    params =  getBooster()
    model = LGBMRegressor(params)
    model_co = model.fit(X,y)
    preds=model_co.predict(X)
    model_dict[i]=model
    preds_dict[i]=preds

Mem. usage decreased to 91.06 Mb (49.0% reduction)
Mem. usage decreased to  0.00 Mb (56.9% reduction)
Mem. usage decreased to  0.00 Mb (57.1% reduction)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

Best trial:
  Params: 
    feature_fraction: 0.537213475330407
    min_gain_to_split: 0.41620873567348476
    max_depth: 87
    lambda_l1: 6.735462589674785e-08
    lambda_l2: 2.792702901126982e-07
    num_leaves: 184
    num_iterations: 392
    learning_rate: 0.0016710567462247096
Mem. usage decreased to 93.83 Mb (46.9% reduction)
Mem. usage decreased to  0.00 Mb (45.0% reduction)
Mem. usage decreased to  0.01 Mb (0.0% reduction)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

Best trial:
  Params: 
    feature_fraction: 0.5343083254533079
    min_gain_to_split: 0.7686496111368774
    max_depth: 49
    lambda_l1: 0.5107145736360436
    lambda_l2: 0.00621400401635028
    num_leaves: 75
    num_iterations: 269
    learning_rate: 0.06442120613776209
Mem. usage decreased to 95.13 Mb (46.9% reduction)
Mem. usage decreased to  0.01 Mb (0.0% reduction)
Mem. usage decreased to  0.01 Mb (0.0% reduction)


...

Best trial:
  Params: 
    feature_fraction: 0.6243816616102809
    min_gain_to_split: 0.3014329203176722
    max_depth: 64
    lambda_l1: 0.24974039005480786
    lambda_l2: 0.02692015498685569
    num_leaves: 200
    num_iterations: 375
    learning_rate: 0.009831965330615876
Mem. usage decreased to 29.46 Mb (49.0% reduction)
Mem. usage decreased to  0.01 Mb (0.0% reduction)
Mem. usage decreased to  0.01 Mb (0.0% reduction)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

Best trial:
  Params: 
    feature_fraction: 0.6129797752317567
    min_gain_to_split: 0.54808658149606
    max_depth: 77
    lambda_l1: 1.181448571813902e-07
    lambda_l2: 6.192287025373575e-07
    num_leaves: 137
    num_iterations: 285
    learning_rate: 0.08719776550883525
Mem. usage decreased to 51.15 Mb (44.8% reduction)
Mem. usage decreased to  0.01 Mb (0.0% reduction)
Mem. usage decreased to  0.01 Mb (0.0% reduction)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

Best trial:
  Params: 
    feature_fraction: 0.6692169491406683
    min_gain_to_split: 0.9819455290213726
    max_depth: 83
    lambda_l1: 3.932774001035238e-08
    lambda_l2: 0.36682725332440197
    num_leaves: 198
    num_iterations: 118
    learning_rate: 0.028630165826050415


In [16]:
import gresearch_crypto
env = gresearch_crypto.make_env()
iter_test = env.iter_test()

for i, (df_test, df_pred) in enumerate(iter_test):
    print(i)
    for j , row in df_test.iterrows():        
        if model_dict[row['Asset_ID']] is not None:
            try:
                model = model_dict[row['Asset_ID']]
                x_test = get_features(row)
                y_pred = model.predict(pd.DataFrame([X_test]))[0]
                df_pred.loc[df_pred['row_id'] == row['row_id'], 'Target'] = y_pred
            except:
                df_pred.loc[df_pred['row_id'] == row['row_id'], 'Target'] = 0
#                 traceback.print_exc()
        else: 
            df_pred.loc[df_pred['row_id'] == row['row_id'], 'Target'] = 0  
    
    env.predict(df_pred)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
0
1
2
3
